In [8]:
# STEP 1
# Goal: Reading the CSV File, Find ecDNA in the CSV File => list of cycle files name

# Imports
import pandas as pd

# Variables
filePath = "~/Downloads/results/"
fileName = "aggregated_results.csv"
suffixOfCycleFileName = "_annotated_cycles.txt"
cycleFileNames = []

# Filters rows that have the classification 'ecDNA'
dataTable = pd.read_csv(filePath + fileName)
dataTable = dataTable[dataTable['Classification'].notna()]
dataTable = dataTable[dataTable['Classification'].str.contains('ecDNA')]

dataTable = dataTable['Feature ID']

# Makes a list of cycle file names
for id in dataTable:
    index = id.index('amplicon')
    cycleFileNames.append(id[:index+9] + suffixOfCycleFileName)

print(cycleFileNames)

['ESO51_amplicon4_annotated_cycles.txt', 'ESO51_amplicon9_annotated_cycles.txt', 'JH-EsoAd1_amplicon3_annotated_cycles.txt', 'JH-EsoAd1_amplicon4_annotated_cycles.txt', 'JH-EsoAd1_amplicon5_annotated_cycles.txt', 'JH-EsoAd1_amplicon5_annotated_cycles.txt', 'JH-EsoAd1_amplicon5_annotated_cycles.txt', 'FLO-1_amplicon3_annotated_cycles.txt', 'FLO-1_amplicon4_annotated_cycles.txt', 'OACP4-C_amplicon1_annotated_cycles.txt', 'OACP4-C_amplicon1_annotated_cycles.txt', 'OACP4-C_amplicon1_annotated_cycles.txt', 'OACP4-C_amplicon1_annotated_cycles.txt', 'OACP4-C_amplicon1_annotated_cycles.txt', 'OACP4-C_amplicon1_annotated_cycles.txt', 'OACP4-C_amplicon1_annotated_cycles.txt', 'OACP4-C_amplicon2_annotated_cycles.txt', 'OACP4-C_amplicon2_annotated_cycles.txt', 'OACP4-C_amplicon2_annotated_cycles.txt', 'OACP4-C_amplicon2_annotated_cycles.txt', 'OACP4-C_amplicon2_annotated_cycles.txt', 'OACP4-C_amplicon2_annotated_cycles.txt', 'OACP4-C_amplicon3_annotated_cycles.txt', 'OACP4-C_amplicon4_annotated_cy

In [7]:
# STEP 2
# Goal: Open the Feature ID under cycles.txt file, find the cycles in corresponding cycles.txt file 
# => file with segments & only actual cycle content

df = pd.DataFrame(columns=['sample', 'segment', 'chr', 'start_pos', 'end_pos'])
for file_name in cycleFileNames:
    sample_name = file_name.split('_')[0]
    path = f'../Downloads/results/AA_outputs/{sample_name}/{sample_name}_classification/{sample_name}_annotated_cycles_files/{file_name}'

    with open(path) as file:

        # skip lines until cycle info 
        file.readline() # skip title 
        line = file.readline()
        while line[0] == 'S':
            line = file.readline()
        
        # find segments for cycles
        segments = set()
        while line:
            sep_line = line.split(';')
            cyclic = sep_line[3]
            cyclic_bool = cyclic.split('=')[1]
            if cyclic_bool == 'True':
                cycle_segments = sep_line[-1].split('=')[1]
                cycle_segments = cycle_segments.split(',')
                for i in range(len(cycle_segments)):
                    cycle_segments[i] = cycle_segments[i].strip('+-\n')
                for seg in cycle_segments:
                    segments.add(seg)
            line = file.readline()

        # identify chr positions for cyclic segments 
        file.seek(0)
        file.readline() # skip title
        line = file.readline()
        while line[0] == 'S':
            sep_line = line.split('\t')
            if sep_line[1] in segments:
                df.loc[len(df.index)] = [sample_name, sep_line[1], sep_line[2], sep_line[3], sep_line[4].strip()]
            line = file.readline()
        
display(df)

,sample,segment,chr,start_pos,end_pos
0,ESO51,5,chr7,111079423,111153919
1,ESO51,6,chr7,111081597,111152189
2,ESO51,2,chr20,49132929,54270699
3,ESO51,6,chr20,49138754,54297673
4,JH-EsoAd1,4,chr11,92522622,92603787
...,...,...,...,...,...
172,OE33,15,chr7,114446989,115264904
173,OE33,18,chr7,115234758,120482720
174,OE33,20,chr12,107809481,107809486
175,OE33,5,chr17,29187871,29854741
